## Transforming Data

In this section, we'll transform the data to make it ready for algorithm as correct as possible (check Readme for details).

In [1]:
import pandas as pd

#minimum number of games played (firest eleven or substituted on) for a player to be evaluated
#players who has played less then this number will be removed from data
min_games = 10

In [2]:
#read player data from csv file
player_stats_df = pd.read_csv("data/player_stats.csv", index_col=0)

First, non-numeric attributes will be dropped and then the data of transferred players will be separated

In [3]:
#get non-numeric attributes for the players. we'll use this player info data later to identify players 
players_df = player_stats_df[["player_id", "player_name", "team", "age", "country",
                             "avg_rating", "height_cm","position", "weight_kg",
                              "first_eleven", "substituted_on", "minutes_played"]].copy()

#remove goalkeepers from the data since most of the stats are related to outfield players
player_stats_df = player_stats_df.loc[player_stats_df["position"] != "GK"]
player_stats_df = player_stats_df.loc[player_stats_df["position"] != "Goalkeeper"]

#calculate total number of the games played by each player either as first eleven or being substituted on
player_stats_df["games_played"] = player_stats_df["first_eleven"]+player_stats_df["substituted_on"]

#drop non-numeric attributes from data. we already saved these in another dataframe
player_stats_df.drop(["team", "age", "avg_rating", "country", "first_eleven", "height_cm",
                      "minutes_played", "player_name", "position", "substituted_on",
                      "weight_kg"], axis=1, inplace=True)

#number of ocurences of players in data
#to identify players who transferred between teams during mid-season transfer window
#stats for those players will be aggregated accordigly
counts_df = player_stats_df.groupby(["player_id"]).size().reset_index(name="counts")

#add count of occurences to the dataframe as a new column
player_stats_df = player_stats_df.merge(counts_df, on="player_id")

#get trasferred players into a new dataframe
transferred_df = player_stats_df.loc[player_stats_df["counts"] > 1]

#remove transferred players. we'll add them after some calculations
player_stats_df = player_stats_df.loc[player_stats_df["counts"] < 2]

#drop counts column since it's not necessary in this data
player_stats_df.drop(["counts"], axis=1, inplace=True)

Aggregate data for transferred players

In [4]:
#unlike other attributes, pass accuracy is a percentage
#we need to calculate total number of accurate passes in order to aggregate pass accuracy correctly 
transferred_df["accurate_passes"] = transferred_df["pass_accuracy"]*transferred_df["passes"]*transferred_df["games_played"]/100

#drop pass accuracy. it'll be re-calculated
transferred_df.drop(["pass_accuracy", "counts"], axis=1, inplace=True)

#some numbers are per game averages whilst some numbers are total occurrences in all the games played
#per games numbers should be multiplied with the number of games played
#in order to aggreagete those numbers correctly
averages_df = transferred_df[["aerials_won", "bad_controls", "blocks", "clearances", "crosses",
               "dispossessed", "dribbled_past", "dribbles", "fouled", "fouls",
               "interceptions", "key_passes", "long_balls", "offsides", "offsides_won",
               "passes", "shots", "tackles", "through_balls"]].copy()

#drop per game numbers. we'll add them after calculation.
transferred_df.drop(["aerials_won", "bad_controls", "blocks", "clearances", "crosses",
               "dispossessed", "dribbled_past", "dribbles", "fouled", "fouls",
               "interceptions", "key_passes", "long_balls", "offsides", "offsides_won",
               "passes", "shots", "tackles", "through_balls"], axis=1, inplace=True)

#multiply per game numbers with the number of games played to get total number of occurrences
averages_df = averages_df.mul(transferred_df["games_played"], axis=0)

#we don't want to multiply player ids, so added it after multiplication
averages_df["player_id"] = transferred_df["player_id"]

#now aggreagate, then drop player ids since we don't want to divide those ids
averages_df = averages_df.groupby(["player_id"], as_index=False).sum()
player_ids = averages_df["player_id"]
averages_df.drop(["player_id"], axis=1, inplace=True)

#aggregate total numbers
transferred_df = transferred_df.groupby(["player_id"], as_index=False).sum()

#re-calculate averages by games played and then add player ids
averages_df = averages_df.div(transferred_df["games_played"], axis=0)
averages_df["player_id"] = player_ids

#merge dataframes
transferred_df = transferred_df.merge(averages_df, on="player_id")

#lastly, calculate pass accuracy as percentage and then we're good to go
transferred_df["pass_accuracy"] = 100*transferred_df["accurate_passes"]/(transferred_df["passes"]*transferred_df["games_played"])
transferred_df.drop(["accurate_passes"], axis=1, inplace=True)

Add aggregated data to base dataframe and then do the normalization

In [5]:
#add re-calculated transferred players
player_stats_df = player_stats_df.append(transferred_df)

#change all the stats with total number of occurrences to per game averages
player_stats_df["assists"] = player_stats_df["assists"]/player_stats_df["games_played"]
player_stats_df["goals"] = player_stats_df["goals"]/player_stats_df["games_played"]
player_stats_df["man_of_the_match"] = player_stats_df["man_of_the_match"]/player_stats_df["games_played"]
player_stats_df["own_goals"] = player_stats_df["own_goals"]/player_stats_df["games_played"]
player_stats_df["red_cards"] = player_stats_df["red_cards"]/player_stats_df["games_played"]
player_stats_df["yellow_cards"] = player_stats_df["yellow_cards"]/player_stats_df["games_played"]

#remove the players who didn't play at least given minimum number of games
player_stats_df = player_stats_df.loc[player_stats_df["games_played"] >= min_games]

#we don't need number of games played anymore. it has been already saved in player info dataframe
player_stats_df.drop(["games_played"], axis=1, inplace=True)

#after adding transferred playersi there are some duplicate indexes, so we'll reset them
player_stats_df.reset_index(drop=True, inplace=True)

In [6]:
#remove the bias in data (check "transforming data" section in readme)
for column in player_stats_df:
    if column != "player_id":
        player_stats_df[column] = player_stats_df[column] / player_stats_df[column].max()

Save data for stats and player information

In [7]:
#save dataframes
players_df.to_csv("data/players.csv", encoding="utf-8")
player_stats_df.to_csv("data/stats.csv", encoding="utf-8")